# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [1]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [2]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2511856
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [3]:
import os
import pandas as pd

os.mkdir('./outputs')

Now, let's load some more packages we will need later:

In [4]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [5]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/320 [00:00<?, ? examples/s]

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

/Users/wukehui/anaconda3/envs/text_analytics/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [7]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)

    ### WRITE YOUR ANSWER HERE
    
    # Count occurrences of each unique class in the labels
    unique_classes, class_counts = np.unique(labels, return_counts=True)

    # Compute prior probability for each class
    priors[unique_classes] = class_counts / len(labels)

    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.33875 0.41125 0.25   ]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [8]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    
    # Count occurrences of each feature for each class
    for c in range(num_classes):
        class_indices = np.where(labels == c)[0]  # Get indices of documents in class c
        class_feature_counts = X[class_indices].sum(axis=0)  # Sum feature occurrences for class c
        
        # Compute likelihood with Laplace smoothing
        likelihoods[:, c] = (class_feature_counts + 1) / (class_feature_counts.sum() + num_features)

    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)

### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [9]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-1.27982614e+01 -1.02745411e+00 -4.43042037e-01]
 [-2.10852069e+01 -2.30081127e-08 -1.76181503e+01]
 [-7.32489993e+00 -1.80622984e-01 -1.80429130e+00]
 [-1.83669289e+01 -1.05526397e-08 -2.84852121e+01]
 [-1.17275924e+01 -8.18345666e-06 -1.59755353e+01]
 [-6.82587414e-02 -2.71838502e+00 -2.47279616e+01]
 [-4.67698027e+00 -1.81842040e-02 -4.74296293e+00]
 [-1.55264462e+01 -8.74151427e+00 -1.60005203e-04]
 [-3.02400984e-02 -3.59329065e+00 -6.08367730e+00]
 [-1.70696126e-06 -1.36000349e+01 -1.45779907e+01]
 [-8.56309138e-01 -5.53382841e-01 -8.21036700e+00]
 [-4.93081178e-01 -9.43625944e-01 -1.00243883e+01]
 [-1.15514906e+01 -9.62175949e-06 -2.45140982e+01]
 [-1.24403294e-02 -4.39448928e+00 -1.09204464e+01]
 [-2.99636291e+01 -1.13686838e-13 -4.18797924e+01]
 [-4.27751107e+01 -1.08040882e-03 -6.83095593e+00]
 [-4.03653227e-01 -1.10536768e+00 -6.87316762e+00]
 [-2.31110916e-02 -3.80235457e+00 -7.54657411e+00]
 [-4.50612765e+00 -1.12096210e-02 -9.15306607e+00]
 [-8.24689041e+00 -1.23855060e+

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.75

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [11]:
top_features = []

### WRITE YOUR ANSWER HERE

# Get the log-likelihoods for class 0
log_likelihoods_class_0 = np.log(likelihoods[:, 0])

# Get indices of the top 5 features with highest log-likelihood for class 0
top_feature_indices = np.argsort(log_likelihoods_class_0)[-5:]  # Get top 5 indices

# Map indices back to feature names using the vectorizer
top_features = [vectorizer.get_feature_names_out()[i] for i in top_feature_indices]

### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['to the', 'in the', ', and', 'of the', 'climate change']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE: Version 1

    vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),  # unigrams + bigrams
        stop_words='english',
        lowercase=True,
        min_df=2,           # ignore very rare words
        max_df=0.95,        # ignore extremely frequent terms (e.g. "climate")
        token_pattern=r'\b\w\w+\b'  # tokens with >=2 chars
    )

    X = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)
    num_features = X.shape[1]
    
    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

0.755


In [13]:
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE: Version 3

    vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),
        stop_words='english',
        lowercase=True,
        max_features=10000,  # limit dimensions for regularisation
        min_df=2,
        max_df=0.95,
    )

    X = vectorizer.fit_transform(train_texts)
    X_val = vectorizer.transform(val_texts)
    num_features = X.shape[1]
    
    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

0.755


In [14]:
import re

def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE: Version 2
    
    def clean_text(text):
        text = text.lower()
        text = re.sub(r'\d+', '', text)  # remove digits
        text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
        text = re.sub(r'\s+', ' ', text)  # remove extra spaces
        return text.strip()

    cleaned_train = [clean_text(t) for t in train_texts]
    cleaned_val = [clean_text(t) for t in val_texts]

    vectorizer = TfidfVectorizer(
        ngram_range=(1, 2),
        stop_words='english',
        min_df=2,
        max_df=0.95,
    )

    X = vectorizer.fit_transform(cleaned_train)
    X_val = vectorizer.transform(cleaned_val)
    num_features = X.shape[1]
    
    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

0.76


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [15]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL

from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from torch import nn, optim
import torch
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

# Hyperparameters
sequence_length = 128
batch_size = 16
learning_rate = 2e-5  # Lower learning rate for fine-tuning transformers
num_epochs = 6  # Slightly increased epochs

# Tokenizer setup
tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=sequence_length,
        return_tensors="pt"
    )

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = val_dataset.map(tokenize_function, batched=True)
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define classifier
class BERTClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("climatebert/distilroberta-base-climate-f")
        self.dropout = nn.Dropout(0.2)  # Reduced dropout to retain more signal
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
        x = self.dropout(cls_output)
        return self.classifier(x)

# Initialize model
model = BERTClassifier(num_classes=num_classes).to(device)

# Training function
def train(model, train_loader, val_loader, epochs, lr):
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_losses = []

        for batch in train_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device).long()

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {np.mean(train_losses):.4f}")

        # Validation step
        model.eval()
        val_losses = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["label"].to(device).long()

                outputs = model(input_ids, attention_mask)
                val_loss = loss_fn(outputs, labels)
                val_losses.append(val_loss.item())

        print(f"Epoch {epoch+1}/{epochs} - Val Loss: {np.mean(val_losses):.4f}")

# Train the model
train(model, train_loader, val_loader, num_epochs, learning_rate)

# Predict function
def predict(model, data_loader):
    model.eval()
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            outputs = model(input_ids, attention_mask)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())

    return all_preds

predictions_nn = predict(model, val_loader)

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Some weights of RobertaModel were not initialized from the model checkpoint at climatebert/distilroberta-base-climate-f and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6 - Train Loss: 0.8968
Epoch 1/6 - Val Loss: 0.4856
Epoch 2/6 - Train Loss: 0.3916
Epoch 2/6 - Val Loss: 0.4420
Epoch 3/6 - Train Loss: 0.2347
Epoch 3/6 - Val Loss: 0.4898
Epoch 4/6 - Train Loss: 0.1285
Epoch 4/6 - Val Loss: 0.5149
Epoch 5/6 - Train Loss: 0.0545
Epoch 5/6 - Val Loss: 0.6471
Epoch 6/6 - Train Loss: 0.0359
Epoch 6/6 - Val Loss: 0.6114
0.835


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [16]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [17]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoModel, AutoTokenizer

# Load the pretrained BERT-tiny model and tokenizer
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Tokenize the chosen document
inputs = tokenizer(chosen_document, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

# Extract input tensors
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Pass the input through the model to obtain embeddings
with torch.no_grad():  # Disable gradient calculation for efficiency
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)

# Extract the embedding of the [CLS] token (first token representation)
doc_emb = outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Convert tensor to NumPy array


### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###

[ 1.2353779   1.2661054  -4.3272057  -0.56385696 -0.4071368   0.7104719
  0.572997    1.6965109  -0.41146696  0.67622983  0.56909686  0.7041474
  0.6614523  -0.02387507  1.868561    0.10236161 -1.3474936  -1.1051005
 -0.12682438  1.6499944   0.57981694  0.60964864  1.8799553  -0.19296098
  1.3707212  -1.288001   -0.50483584  0.45296505 -0.6050616  -0.03914492
  0.47469813 -0.7097695  -1.2004123  -0.10839067  0.6036101  -1.8727187
 -0.93857    -0.11339553 -2.208518   -1.1077261   0.07236955  0.7870133
  0.04916956 -2.6308413   0.26365525 -2.8721178  -0.14297323 -0.06168235
 -0.5415106   0.8062427   0.92474943  2.2324717  -0.41735238 -0.02618527
 -0.8402096   0.22188668 -0.91551745 -0.8256168   0.74066985  2.6733189
  0.63273907  0.89870435 -2.8598394  -1.5780993  -1.2796944   0.41822475
  0.25070715  0.37491843 -0.80434376 -1.3791412  -0.6677291  -0.30917653
  0.8928048   0.36302948 -1.1116061   0.31267783  1.018613   -0.06540625
  0.7463695   0.82981306 -0.5589379  -0.59806204 -0.57203

## TASK 1.3b

Using the same document embeddings method as the previous task (1.3a), find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors. (2 marks)

In [18]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModel, AutoTokenizer

# Load the BERT-Tiny model and tokenizer
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    """Function to get the [CLS] token embedding for a given text."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # Extract [CLS] embedding

# Get embedding for the chosen document
chosen_embedding = get_embedding(chosen_document)

# Compute embeddings for all validation documents
val_embeddings = np.array([get_embedding(doc) for doc in val_texts])

# Compute cosine similarity between chosen_document and each validation document
similarities = cosine_similarity([chosen_embedding], val_embeddings)[0]

# Find the index of the most similar document
most_similar_index = np.argmax(similarities)
most_similar_doc = val_texts[most_similar_index]

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###

In May 2020, BBVA was the first private financial institution in Europe to issue a COVID-19-related social bond and, two months later, the Bank was the first financial institution to issue contingent convertible perpetual bonds (CoCos) classified as green bonds, for EUR 1 billion. The funds will be used to finance eligible green assets in BBVA's portfolio. The portfolio is diversified into assets from different green sectors: energy efficiency, renewable energy, sustainable transportation, waste management and water management.
ING issued a dual tranche five-year EUR 500 million and three-year USD 800 million green bond. This is ING’s first-ever green bond transaction. The money raised will go to projects in six categories eligible under ING’s newly established green bond framework, including renewable energy, green buildings, public transport, waste, water and energy efficiency. We have chosen a broad selection of sectors, which reflects our ambition to support sustainability across a

## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [19]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import torch

# Load the pretrained BERT-Tiny model and tokenizer
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Load the dataset
train_dataset = load_dataset("climatebert/climate_sentiment", split="train")
val_dataset = load_dataset("climatebert/climate_sentiment", split="test")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Apply the tokenization to both train and validation datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Set the format to PyTorch tensors
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# Define the compute_metrics function
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# Setup the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",  # Evaluation strategy after each epoch
    save_strategy="epoch",  # Save strategy after each epoch
    logging_dir="./logs",
    logging_steps=10,
    save_steps=1000,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model based on evaluation
    metric_for_best_model="accuracy"
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Predict on the validation set
predictions_bert = trainer.predict(val_dataset).predictions.argmax(axis=-1)

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

/Users/wukehui/anaconda3/envs/text_analytics/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.085400,1.033014,0.521875
2,1.051600,1.009986,0.515625
3,1.021300,1.005229,0.518750


0.521875
